In [1]:
import pickle
import numpy as np
import scipy 
import torch
import matplotlib.pyplot as plt

DD_mat = torch.load('../data/DD_mat_IC_L.pt')


In [ ]:
with open('../experiments/FNN-DD_IC-3000-B128-lr0.0001-IC-4/val_pred.pkl', 'rb') as f:
    data = pickle.load(f)

print(data.keys())

L_IC = data['true'][0]
pred_ep = data['pred'][0]
inputs = data['inputs'][0]
print(inputs.shape)

print(DD_mat.keys())
print(L_IC.shape)
print(pred_ep.shape)


# put the entries back to the matrix
DD = DD_mat['A'][0]
IC = DD_mat['L_IC'][0]
mask = torch.abs(DD) != 0
mask_ic = torch.abs(IC) != 0

# mask = mask.repeat(pred_ep.shape[0], 1, 1)
# mask_ic = mask_ic.repeat(pred_ep.shape[0], 1, 1)
print(DD.shape)
print(L_IC.dtype)
DD_input = torch.zeros_like(DD)
DD_input = DD_input[None, :, :].repeat(pred_ep.shape[0], 1, 1)
print(DD_input.shape)
repeated_tensors = [DD[None, :, :] for _ in range(pred_ep.shape[0])]
DD_input = torch.cat(repeated_tensors, dim=0)
assert False
ic_placeholder = torch.zeros_like(DD, dtype=L_IC.dtype).repeat(pred_ep.shape[0], 1, 1)
ep_placeholder = torch.zeros_like(DD, dtype=L_IC.dtype).repeat(pred_ep.shape[0], 1, 1)


DD_input[:, mask] = data['inputs'][0]
ic_placeholder[:, mask_ic] = L_IC
ep_placeholder[:, mask_ic] = pred_ep

L_IC = ic_placeholder
pred_ep = ep_placeholder

# construct L when predicting the difference
# L = I + alpah * eplsion
#I = torch.eye(128).repeat(pred_ep.shape[0], 1, 1)
#L_pred = I + data['scale'] * pred_ep
L_pred =  pred_ep
print(L_pred.shape)

In [ ]:
# get preconditioner
# the prediction not need to be inverted
LL_pred = torch.bmm(L_pred, L_pred.conj().transpose(-2, -1))

LL_IC = torch.bmm(L_IC, L_IC.conj().transpose(-2, -1))
LL_IC_inv = torch.linalg.inv(LL_IC)

LLDD_pred = torch.bmm(LL_pred, DD_input)
LLDD_IC = torch.bmm(LL_IC_inv, DD_input)


# compare the condition numbers
test_id = 1
cond_DD = np.linalg.cond(DD_input[test_id])
print('condition number of the original system', cond_DD)
cond_pc = np.linalg.cond(LLDD_pred[test_id])
print('condition number of the PC system', cond_pc)
cond_pc_true = np.linalg.cond(LLDD_IC[test_id])
print('condition number of the PC system from IC', cond_pc_true)

In [ ]:
org_cond = []
L_pc_cond = []
IC_pc_cond = []

for i in range(DD_input.shape[0]):
    org_cond.append(np.linalg.cond(DD_input[i]))
    L_pc_cond.append(np.linalg.cond(LLDD_pred[i]))
    IC_pc_cond.append(np.linalg.cond(LLDD_IC[i]))

org_cond = np.array(org_cond)
L_pc_cond = np.array(L_pc_cond)
IC_pc_cond = np.array(IC_pc_cond)
sorted_idx = np.argsort(org_cond)

org_cond = org_cond[sorted_idx]
L_pc_cond = L_pc_cond[sorted_idx]
IC_pc_cond = IC_pc_cond[sorted_idx]
x = np.arange(org_cond.shape[0])

fig, ax = plt.subplots()
ax.set_box_aspect(1/1.62)
ax.scatter(x, org_cond, label='Original')
ax.scatter(x, L_pc_cond, label='NN precond')
ax.scatter(x, IC_pc_cond, label='IC precond')
ax.set_xlabel('Sample ID')
ax.set_ylabel('Condition number')
ax.set_ylim(0, 1000)
#ax.set_yscale('log')
ax.legend()
ax.grid(linestyle='dotted')
plt.show()

In [ ]:
fig.savefig('../../docs/updates/figs/NN-epsilon-L-condition-number-compare.png', format='png', dpi=500, bbox_inches='tight')


In [ ]:
# solve using CG
from scipy.sparse.linalg import cg

import time

def cg_solve(batch_A, precond=None):
    residual_list = []
    iter_time_list = []
    for i in range(batch_A.shape[0]):
        residuals = []
        iter_time = []
        b = np.random.rand(128) + 1j*np.random.rand(128)
        curr_time = time.time()
        def store_residual(xk):
            residual = np.linalg.norm(b - batch_A[i].numpy()@xk)
            iter_time.append(time.time() - curr_time)
            residuals.append(residual)
        if precond is not None:
            M = precond[i]
        else:
            M = None
        x = cg(A=batch_A[i].numpy(), 
               M=M, 
               b=b, 
               tol=1e-20, 
               maxiter=80,
               callback=store_residual)
        residual_list.append(residuals)
        iter_time_list.append(iter_time)
    return residual_list, iter_time_list

residual_DD, time_DD = cg_solve(DD_input)
residual_DD_L, time_L = cg_solve(DD_input, precond=LL_pred.numpy())
residual_DD_IC, time_IC = cg_solve(DD_input, precond=LL_IC_inv.numpy())

In [ ]:
idx = sorted_idx[-1]
fig, ax = plt.subplots()
ax.set_box_aspect(1/1.62)
ax.plot(residual_DD[idx], label='Original')
ax.plot(residual_DD_L[idx], label='NN precond')
ax.plot(residual_DD_IC[idx], label='IC precond')
ax.grid(linestyle='dotted')
ax.legend()

ax.set_yscale('log')
ax.set_xlabel('Iterations')
ax.set_ylabel('Residual norm')

In [ ]:
# take the average and standard deviation over all instances
# take the first 30 steps
steps = 80
residual_DD_30 = np.array([r[:steps] for r in residual_DD])
residual_DD_L_30 = np.array([r[:steps] for r in residual_DD_L])
residual_DD_IC_30 = np.array([r[:steps] for r in residual_DD_IC])

resi_DD_mean = np.mean(residual_DD_30, axis=0)
resi_DD_L_mean = np.mean(residual_DD_L_30, axis=0)
resi_DD_IC_mean = np.mean(residual_DD_IC_30, axis=0)

resi_DD_std = np.std(residual_DD_30, axis=0)
resi_DD_L_std = np.std(residual_DD_L_30, axis=0)
resi_DD_IC_std = np.std(residual_DD_IC_30, axis=0)

fig, ax = plt.subplots()
ax.set_box_aspect(1/1.62)
ax.plot(resi_DD_mean, label='Original', color='b')
ax.fill_between(np.arange(len(resi_DD_std)), resi_DD_mean - resi_DD_std, resi_DD_mean + resi_DD_std, alpha=.08, color='b')
ax.plot(resi_DD_L_mean, label='NN precond', color='orange')
ax.fill_between(np.arange(len(resi_DD_L_std)), resi_DD_L_mean - resi_DD_L_std, resi_DD_L_mean + resi_DD_L_std, alpha=.08, color='orange')
ax.plot(resi_DD_IC_mean, label='IC precond', color='green')
ax.fill_between(np.arange(len(resi_DD_IC_std)), resi_DD_IC_mean - resi_DD_IC_std, resi_DD_IC_mean + resi_DD_IC_std, alpha=.08, color='green')
ax.grid(linestyle='dotted')
ax.legend()

ax.set_yscale('log')
ax.set_xlabel('Iterations')
ax.set_ylabel('Residual norm')
fig.savefig('../../docs/updates/figs/NN-epsilon-L-CGsolve-compare.png', format='png', dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(time_DD[0], residual_DD[0])
plt.plot(time_L[0], residual_DD_L[0])
plt.plot(time_IC[0], residual_DD_IC[0])
plt.yscale('log')

# can plot something like reaching 1e-4 takes on average [] seconds
# this should be a barplot
# need to get the index of the first residual that is smaller than 1e-4
print(len(time_DD[1]))
def find_index(time_list, resi_list, cutoff=1e-4):
    j = 0
    for r in resi_list:
        if r < cutoff:
            break
        else:
            pass
        j += 1
    return time_list[j]


cutoff_time_DD = []
cutoff_time_L = []
cutoff_time_IC = []

for i in range(len(time_DD[0])):
    cutoff_time_DD.append(find_index(time_DD[i], residual_DD[i]))
    cutoff_time_L.append(find_index(time_L[i], residual_DD_L[i]))
    cutoff_time_IC.append(find_index(time_IC[i], residual_DD_IC[i]))

idx_DD_mean = np.mean(cutoff_time_DD, axis=0)
idx_DD_std = np.std(cutoff_time_DD, axis=0)

idx_L_mean = np.mean(cutoff_time_L, axis=0)
idx_L_std = np.std(cutoff_time_L, axis=0)

idx_IC_mean = np.mean(cutoff_time_IC, axis=0)
idx_IC_std = np.std(cutoff_time_IC, axis=0)


fig, ax = plt.subplots()
ax.set_box_aspect(1/1.62)
ax.bar(np.arange(3), np.array([idx_DD_mean, idx_L_mean, idx_IC_mean]), 
       yerr=[idx_DD_std, idx_L_std, idx_IC_std],
       width=.5, color=['b', 'orange', 'g'])
ax.set_xticks([0, 1, 2])
ax.set_xticklabels(['Original', 'NN precond', 'IC precond'])
ax.set_ylabel('Seconds')
ax.grid(linestyle='dotted')

fig.savefig('../../docs/updates/figs/NN-epsilon-L-CGsolve-time-compare.png', format='png', dpi=500, bbox_inches='tight')